# CIS 194: Homework 7

In [1]:
import Data.Monoid
:l Sized
:l Buffer
:l Editor

data JoinList m a
  = Empty
  | Single m a
  | Append m (JoinList m a) (JoinList m a)
  deriving (Eq, Show)

## Exercise 1

We first consider how to write some simple operations
on these `JoinLists`. Perhaps the most important operation we will
consider is how to append two JoinLists. Previously, we said that
the point of `JoinLists` is to represent append operations as data, but
what about the annotations? Write an append function for `JoinLists`
that yields a new `JoinList` whose monoidal annotation is derived
from those of the two arguments.

```haskell
(+++) :: Monoid m => JoinList m a -> JoinList m a -> JoinList m a
```

In [2]:
tag :: (Monoid m) => JoinList m a -> m
tag (Append m _ _) = m
tag (Single m _) = m
tag Empty = mempty

(+++) :: (Monoid m) => JoinList m a -> JoinList m a -> JoinList m a
(+++) Empty Empty = Empty
(+++) x Empty = x
(+++) Empty y = y
(+++) x y = Append res x y
  where
    res = tag x <> tag y

In [3]:
exLeft = Single (Product 5) 'y'
exRight = Append (Product 6) (Single (Product 2) 'e') (Single (Product 3) 'a')
exLeft +++ exRight

Append (Product {getProduct = 30}) (Single (Product {getProduct = 5}) 'y') (Append (Product {getProduct = 6}) (Single (Product {getProduct = 2}) 'e') (Single (Product {getProduct = 3}) 'a'))

## Exercise 2 

The first annotation to try out is one for fast indexing
into a JoinList. The idea is to cache the *size* (number of data elements) of each subtree. This can then be used at each step to determine if the desired index is in the left or the right branch.
We have provided the `Sized` module that defines the `Size` type,
which is simply a `newtype` wrapper around an `Int`. In order to make
`Sizes` more accessible, we have also defined the `Sized` type class
which provides a method for obtaining a `Size` from a value.
Use the `Sized` type class to write the following functions.

1. Implement the function
```haskell
indexJ :: (Sized b, Monoid b) =>
    Int -> JoinList b a -> Maybe a
```

`indexJ` finds the `JoinList` element at the specified index. If the
index is out of bounds, the function returns `Nothing`. By an *index*
in a `JoinList` we mean the index in the list that it represents. That
is, consider a safe list indexing function

```haskell
(!!?) :: [a] -> Int -> Maybe a
[]      !!? _         = Nothing
_       !!? i | i < 0 = Nothing
(x:xs)  !!? 0         = Just x
(x:xs)  !!? i         = xs !!? (i-1)
```

which returns Just the $i$th element in a list (starting at zero) if
such an element exists, or Nothing otherwise. We also consider
an updated function for converting join-lists into lists, just like
`jlbToList` but ignoring the monoidal annotations:

```haskell
jlToList :: JoinList m a -> [a]
jlToList Empty = []
jlToList (Single _ a) = [a]
jlToList (Append _ l1 l2) = jlToList l1 ++ jlToList l2
```

We can now specify the desired behavior of `indexJ`. For any index
$i$ and join-list $jl$, it should be the case that

```hasekll
(indexJ i jl) == (jlToList jl !!? i)
```

That is, calling `indexJ` on a join-list is the same as first converting the join-list to a list and then indexing into the list. The point,
of course, is that indexJ can be more efficient ($O(log n)$ versus
$O(n)$, assuming a balanced join-list), because it gets to use the size
annotations to throw away whole parts of the tree at once, whereas
the list indexing operation has to walk over every element.

In [4]:
len ::
  (Sized b, Monoid b) =>
  JoinList b a -> Int
len = getSize . size . tag

In [5]:
indexJ ::
  (Sized b, Monoid b) =>
  Int -> JoinList b a -> Maybe a
indexJ 0 (Single _ a) = Just a
indexJ i jl@(Append s l r)
  | i < len l = indexJ i l
  | i < len jl = indexJ (i - len l) r
  | otherwise = Nothing
indexJ _ _ = Nothing

In [6]:
jlToList :: JoinList m a -> [a]
jlToList Empty = []
jlToList (Single _ a) = [a]
jlToList (Append _ l1 l2) = jlToList l1 ++ jlToList l2

(!!?) :: [a] -> Int -> Maybe a
[] !!? _ = Nothing
_ !!? i | i < 0 = Nothing
(x : xs) !!? 0 = Just x
(x : xs) !!? i = xs !!? (i - 1)

In [7]:
assumeJl ::
  (Sized b, Monoid b) =>
  (JoinList b a -> Int -> Bool) -> JoinList b a -> Bool
assumeJl f jl
  | len > 0 = all g [0 .. len]
  | otherwise = g 0 && g 1 && g 42
  where
    len = (getSize . size . tag) jl
    g = f jl

assumeIdx ::
  (Sized b, Monoid b, Eq a) =>
  JoinList b a -> Bool
assumeIdx = assumeJl (\jl i -> indexJ i jl == (jlToList jl !!? i))

In [8]:
exLeft = Single (Size 1) 'y'
exRight = Append (Size 2) (Single (Size 1) 'e') (Single (Size 1) 'a')
jl = exLeft +++ exRight
assumeIdx jl

True

2. Implement the function
```haskell
dropJ :: (Sized b, Monoid b) =>
    Int -> JoinList b a -> JoinList b a
```
The `dropJ` function drops the first n elements from a `JoinList`.
This is analogous to the standard `drop` function on lists. Formally,
dropJ should behave in such a way that

```haskell
jlToList (dropJ n jl) == drop n (jlToList jl)
```

In [9]:
dropJ ::
  (Sized b, Monoid b) =>
  Int -> JoinList b a -> JoinList b a
dropJ n jl
  | n <= 0 = jl
  | otherwise = case jl of
      (Append {}) -> f jl
      _ -> Empty
  where
    f jl@(Append s l r)
      | n < len l = dropJ n l
      | n < len jl = dropJ (n - len l) r
      | otherwise = Empty

In [10]:
assumeDrop = assumeJl (\jl i -> jlToList (dropJ i jl) == drop i (jlToList jl))
assumeDrop jl

True

3. Finally, implement the function
```haskell
takeJ :: (Sized b, Monoid b) =>
    Int -> JoinList b a -> JoinList b a
```

The `takeJ` function returns the first n elements of a JoinList,
dropping all other elements. Again, this function works similarly
to the standard library `take` function; that is, it should be the case
that
```haskell
jlToList (takeJ n jl) == take n (jlToList jl)
```

Ensure that your function definitions use the size function from
the Sized type class to make smart decisions about how to descend
into the `JoinList` tree.

In [16]:
takeJ ::
  (Sized b, Monoid b) =>
  Int -> JoinList b a -> JoinList b a
takeJ n jl
  | n <= 0 = Empty
  | otherwise = case jl of
      (Append {}) -> f jl
      _ -> jl
  where
    f jl@(Append s l r)
      | n < len l = takeJ n l
      | n < len jl = (+++) l $ takeJ (n - len l) r
      | otherwise = jl

In [17]:
assumeTake = assumeJl (\jl i -> jlToList (takeJ i jl) == take i (jlToList jl))
assumeTake jl

True

## Exercise 3

Mr. Dickens’s publishing company has changed their
minds. Instead of paying him by the word, they have decided to pay
him according to the scoring metric used by the immensely popular
game of $Scrabble^{TM}$. You must therefore update your editor implementation to count Scrabble scores rather than counting words.

Hence, the second annotation you decide to implement is one
to cache the $Scrabble^{TM}$ score for every line in a buffer. Create a
`Scrabble` module that defines a `Score` type, a `Monoid` instance for
`Score`, and the following functions:

```haskell
score :: Char -> Score
scoreString :: String -> Score
```

The score function should implement the tile scoring values as
shown at http://www.thepixiepit.co.uk/scrabble/rules.html; any
characters not mentioned (punctuation, spaces, *etc*.) should be given
zero points.

To test that you have everything working, add the line `import Scrabble`
to the import section of your `JoinList` module, and write the following function to test out JoinLists annotated with scores:

```haskell
scoreLine :: String -> JoinList Score String
```

Example:
```
*JoinList> scoreLine "yay " +++ scoreLine "haskell!"
Append (Score 23)
(Single (Score 9) "yay ")
(Single (Score 14) "haskell!")
```